# MNIST CLASSIFIER

`MNIST contains 70,000 images of handwritten digits: 60,000 for training and 10,000 for testing. The images are grayscale, 28x28 pixels, and centered to reduce preprocessing and get started quicker.` 

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
n_epochs = 25 #3
batch_size_train = 500 #64
batch_size_test = 1000

learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
#torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [ ]:
'''
import gzip
import numpy as np
import matplotlib.pyplot as plt
import tarfile
import os

data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-03-data'
tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
with tarfile.open(tar_path, mode='r:gz') as tar:
    tar.extractall(path=data_base_path)
def extract_data(filename, image_shape, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(np.prod(image_shape) * image_number)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(image_number, image_shape[0], image_shape[1])
    return data


def extract_labels(filename, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * image_number)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return labels
    '''

In [ ]:
'''
image_shape = (28, 28)
train_set_size = 60000
test_set_size = 10000

data_part2_folder = os.path.join(data_base_path, data_folder, 'part2')

train_images_path = os.path.join(data_part2_folder, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_part2_folder, 'train-labels-idx1-ubyte.gz')
test_images_path = os.path.join(data_part2_folder, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_part2_folder, 't10k-labels-idx1-ubyte.gz')

train_images = extract_data(train_images_path, image_shape, train_set_size)
test_images = extract_data(test_images_path, image_shape, test_set_size)
train_labels = extract_labels(train_labels_path, train_set_size)
test_labels = extract_labels(test_labels_path, test_set_size)
'''

In [ ]:
'''
train_im = train_images.copy()
test_im = test_images.copy()
train_lab = train_labels.copy()
test_lab = test_labels.copy()

train_im = train_im.reshape(train_im.shape[0], 1, train_im.shape[1], train_im.shape[2])
test_im = test_im.reshape(test_im.shape[0], 1, test_im.shape[1], test_im.shape[2])

train_input = torch.from_numpy(train_im)
test_input = torch.from_numpy(test_im)
train_target = torch.from_numpy(train_lab)
test_target = torch.from_numpy(test_lab)

print(train_input.size(), test_input.size(), train_target.size(), test_target.size())
'''

In [ ]:
'''
prng = np.random.RandomState(seed=123456789)  # seed to always re-draw the same distribution
plt_ind = prng.randint(low=0, high=train_set_size, size=10)

fig, axes = plt.subplots(1, 10, figsize=(12, 3))
for ax, im, lb in zip(axes, train_images[plt_ind], train_labels[plt_ind]):
    
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(lb)
    '''

In [6]:
train_transforms = torchvision.transforms.Compose([
    #torchvision.transforms.ToPILImage(),
    #torchvision.transforms.RandomRotation((0,360)),
    #torchvision.transforms.RandomResizedCrop(28),
    #torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,)),
    torchvision.transforms.ToTensor()])

test_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                   torchvision.transforms.Normalize((0.1307,), (0.3081,))])



train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('data/training', train=True, download=True,
                                            transform=train_transforms),
                                            batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('data/test', train=False, download=True,
                                             transform=test_transforms),
                                             batch_size=batch_size_test, shuffle=True)

In [7]:
for data, target in (train_loader):
    data, target = Variable(data), Variable(target)

TypeError: tensor is not a torch image.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,momentum=momentum)

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            #torch.save(network.state_dict(), 'results/model.pth')
            #torch.save(optimizer.state_dict(), 'results/optimizer.pth')

In [ ]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))

In [ ]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

In [ ]:
test_counter

In [ ]:
test_losses

In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')

In [ ]:
with torch.no_grad():
    output = network(example_data)
    
fig = plt.figure()
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title("Prediction: {}".format(
    output.data.max(1, keepdim=True)[1][i].item()))
    plt.xticks([])
    plt.yticks([])

In [ ]:
'''
import torch
from torch import optim, nn
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import torch.nn.functional as F
'''

In [ ]:
'''
# Define a transform to augment the dataset + normalize it
train_transform = transforms.Compose([transforms.RandomRotation((0,360)),
                                      transforms.RandomResizedCrop(28),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5,],[0.5,])])
                                      #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5,],[0.5,])])

train_data = datasets.MNIST('data/training', train=True, download=True, transform=train_transform)
test_data = datasets.MNIST('data/test', train=False, download=True, transform=test_transform)


trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)
'''

In [ ]:
'''
train_data.train_labels
'''

In [ ]:
'''
fig, axes = plt.subplots(4, 6, figsize=(24,16))
ax = axes.ravel()

for ind in range(b.shape[0]):
    
    ax[2*ind].imshow(test_input[a[ind],0], cmap='gray')
    ax[2*ind].axis('off')
    
    width = 0.75
    xpos = np.arange(b.shape[1])  # the x locations for the groups
    ax[2*ind+1].barh(xpos, b[ind], width, color="blue")
    ax[2*ind+1].set_yticks(xpos)
    ax[2*ind+1].set_yticklabels(xpos)
    ax[2*ind+1].set_xlim(0,1)

plt.show()
'''